# Chembl validation pipeline

In [3]:
import os
if not os.getcwd().endswith('src'):
    os.chdir('../src')

# this works with conf file created on 18/11 related to chembl validation:
from conf import TSR_OUT_DRUG, cell_line, pert_time, DISEASE, MITH_IN_DISEASE, MITH_IN_DRUG, map_name_to_id, MITH_APP, MITH_OUT_DRUG, MITH_OUT_DISEASE,\
CS_IN_DRUG, CS_IN_DISEASE, cell_lines_chembl

from validations.chembl.chembl_loader import load_raw_chembl
import numpy as np
from preprocessing_utils import get_drugs_list
import pandas as pd
import pyreadr
import subprocess
from step3_mith_out_to_cs_in_disease import mith_to_cs_in,write_cs_input
from step6_drug_run_MITHrIL import run_mithril_batch
from step7_drug_mith_out_to_cs_in import mith_out_to_cs_in_single_time


print(cell_line, pert_time, DISEASE)

HEPG2 24h LIHC


# Run MIthril on Bin Chen TCGA data

In [2]:
DISEASE = 'LIHC' # todo add dictionary of cell line  to disease for chembl data|

### create Mithril input from TCGA fodl cahnge data

In [40]:
from conf import DATA_DIR

DATA_DIR
tcga = pd.read_excel(DATA_DIR+'BinChen2017'+os.sep+'41467_2017_BFncomms16022_MOESM249_ESM.xlsx', sheet_name = DISEASE+'_sig.csv')
tcga


C:\Users\los4\miniconda3\envs\drug_repurposing\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


,Unnamed: 0,id,baseMean,baseMeanA,baseMeanB,foldChange,log2FoldChange,pval,padj,symbol,GeneID,landmark
0,?|155060,?|155060,246.542053,63.387397,271.093615,4.276775,2.096523,2.322021e-11,4.919731e-10,?,155060,NaN
1,?|653553,?|653553,980.160561,308.154958,1070.241741,3.473064,1.796209,8.513384e-06,7.002229e-05,?,653553,NaN
2,AADAT|51166,AADAT|51166,539.906063,2260.466429,309.267944,0.136816,-2.869691,1.150033e-66,1.149267e-63,AADAT,51166,NaN
3,ABAT|18,ABAT|18,14667.875458,36469.512297,11745.404032,0.322061,-1.634595,1.161602e-36,3.214599e-34,ABAT,18,NaN
4,ABCA8|10351,ABCA8|10351,1142.503781,2728.212260,929.942323,0.340861,-1.552743,4.159067e-18,1.928178e-16,ABCA8,10351,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
1187,ZNF695|57116,ZNF695|57116,18.119918,0.749038,20.448455,27.299608,4.770808,9.450933e-05,5.933800e-04,ZNF695,57116,NaN
1188,ZNF716|441234,ZNF716|441234,12.960449,0.068870,14.688543,213.279970,7.736605,1.291987e-05,1.016635e-04,ZNF716,441234,NaN
1189,ZNF883|169834,ZNF883|169834,41.344740,6.280339,46.045062,7.331620,2.874132,3.883273e-05,2.714820e-04,ZNF883,169834,NaN
1190,ZSWIM5|57643,ZSWIM5|57643,278.072279,76.284806,305.121538,3.999768,1.999916,2.617952e-10,4.834879e-09,ZSWIM5,57643,NaN


In [45]:
fc=tcga[['id','log2FoldChange']]
def split_id(id_line):
    return id_line.split('|')[1]
fc['id']=fc['id'].apply(lambda x : split_id(x))
fc.set_index('id', inplace=True)
fc

C:\Users\los4\AppData\Local\Temp\ipykernel_17252\2076856619.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fc['id']=fc['id'].apply(lambda x : split_id(x))


,log2FoldChange
id,
155060,2.096523
653553,1.796209
51166,-2.869691
18,-1.634595
10351,-1.552743
...,...
57116,4.770808
441234,7.736605
169834,2.874132


In [47]:
#export as mitrhil input
from conf import MITH_IN_DISEASE
fc.to_csv(MITH_IN_DISEASE+DISEASE+'_signature_gene_id.mi', header=None, sep='\t')

In [8]:
from step2_disease_run_MITHrIL import run_mithril, MITH_OUT_DISEASE

In [52]:
run_mithril(DISEASE, MITH_APP, MITH_IN_DISEASE, MITH_OUT_DISEASE)

java -jar /home/signorini/mithril3/app-3.0.0-SNAPSHOT.jar mithril -reactome -p -customize-pathway-matrix -seed 1234 -inversion-factory fast-cpu -multiplication-factory fast-cpu -t 30 -i G:\My Drive\unict\unict 2024-25\netcos\MITHrIL\input\disease_signature\LIHC_signature_gene_id.mi -organism hsa_v2023_03 -o G:\My Drive\unict\unict 2024-25\netcos\MITHrIL\output\disease_signature_2025\LIHC_mith3.output.txt -p G:\My Drive\unict\unict 2024-25\netcos\MITHrIL\output\disease_signature_2025\LIHC_mith3.perturbations.txt -verbose


# Run Drug repurposing for HEPG2_24h

## Convert FC to MITHrIL input for given drug

In [7]:
DRUG_DIR = TSR_OUT_DRUG+'LINCS'+os.sep+cell_line+'_'+pert_time+'_drug_wise'+os.sep
df_list=[]
for i, drug_file in enumerate(os.listdir(DRUG_DIR)):
    drug = drug_file.rstrip('.Rds')
    result = pyreadr.read_r(DRUG_DIR+drug_file)
    print(drug)
    df = result[None] 
    df.set_index('gene_id', inplace=True)
    df.rename(columns = {'DE_log2_FC':drug}, inplace = True)
    
    df_list.append(df[drug])

matrix=pd.concat(df_list, axis=1)
print(matrix.shape)
print(matrix.columns)
print(matrix.head(3))

isonicotinohydroxamic-aci
tozasertib
clomifene
tamoxifen
BRD-A08658758
avrainvillamide-analog-3
vinblastine
IKK-2-inhibitor-V
MW-SHH-54
NSC-632839
MW-SHH-97
BRD-K00640357
diethylstilbestrol
bucladesine
alpha-estradiol
avrainvillamide-analog-1
trichostatin-a
sulforaphane
SIB-1893
equol
estradiol
pentamidine
KI-8751
COL-3
BRD-K75627148
BRD-K67774729
avrainvillamide-analog-4
BRD-K52160314
tyrphostin-AG-879
BRD-K35832492
tert-butylhydroquinone
hydroxyl
HDAC1-selective
podophyllotoxin
staurosporine
AT-9283
simvastatin
BRD-K57064803
malonoben
BRD-K98948170
BRD-K83694683
BRD-K16956545
CGP-53353
KUC111109N
niacin
avrainvillamide-analog-6
A-23187
apicidin
KUC111774N
AT1-SHH-09
crizotinib
BRD-K55896799
BRD-K64106162
tyrphostin-A9
BRD-K61894884
atorvastatin
BRD-K11533227
barasertib-HQPA
estrone
FL-HDAC-025
doxorubicin
arctigenin
avicin-
fulvestrant
BRD-K99636700
raloxifene
benzohydroxamic-aci
MK-5108
spiramide
BRD-K22587354
erastin
sirolimu
Ro-90-7501
pifithrin-mu
BRD-K58547375
BRD-K21001652
ENMD

In [11]:
# Save MITHrIL gene id based input file:
mith_input_filename = 'LINCS_'+cell_line+'_'+pert_time+'.mi'
matrix.to_csv(MITH_IN_DRUG+mith_input_filename, sep='\t', index=True,  index_label=False)
print('MITHrIL drug input saved in',MITH_IN_DRUG,'filename:',LINCS_metanalysis_filename_id)

MITHrIL drug input saved in G:\My Drive\unict\unict 2024-25\netcos\MITHrIL\input\drug_signature\ filename: LINCS_HEPG2_24h.mi


## <b>MITHrIL batch run</b>


In [27]:
MITH_OUT_DRUG = MITH_OUT+'drug_signature_2025_'+cell_line+'_'+pert_time+os.sep
run_mithril_batch(mith_input_filename, MITH_APP, MITH_IN_DRUG, MITH_OUT_DRUG, n_thread="2", printc=True)


java -jar /home/signorini/mithril3/app-3.0.0-SNAPSHOT.jar mithril-batch -reactome -p -customize-pathway-matrix -seed 1234 -inversion-factory fast-cpu -multiplication-factory fast-cpu -t 2 -organism hsa -i G:\My Drive\unict\unict 2024-25\netcos\MITHrIL\input\drug_signature\LINCS_HEPG2_24h.mi -o G:\My Drive\unict\unict 2024-25\netcos\MITHrIL\output\drug_signature_2025_HEPG2_24h\ -p -verbose


# Calculate CS for given TCGA disease and realtive LINCS cell line PF

In [21]:
print(cell_line, pert_time, DISEASE)
print(MITH_OUT_DRUG)
print(MITH_OUT_DISEASE)

HEPG2 24h LIHC
G:\My Drive\unict\unict 2024-25\netcos\MITHrIL\output\drug_signature_2025\HEPG2_24h
G:\My Drive\unict\unict 2024-25\netcos\MITHrIL\output\disease_signature_2025\


## convert drug mith output into cs input

In [2]:
drugs_list=get_drugs_list(True)
print(len(drugs_list))

68


In [3]:
mith_out_to_cs_in_single_time(drugs_list, pert_time)


Mapping mith3 output into tsr connectivity input: 
              Removing pathway duplicates, and filtering 
A-23187 1 of 68
drug A-23187 processed in 0.8
writing mit3 connectivity input metanalysis files for drug A-23187
AT1-SHH-09 2 of 68
drug AT1-SHH-09 processed in 0.8
writing mit3 connectivity input metanalysis files for drug AT1-SHH-09
AVA 3 of 68
drug AVA processed in 0.7
writing mit3 connectivity input metanalysis files for drug AVA
BL003-049-HDAC3 4 of 68
drug BL003-049-HDAC3 processed in 0.7
writing mit3 connectivity input metanalysis files for drug BL003-049-HDAC3
BRD-A08658758 5 of 68
drug BRD-A08658758 processed in 0.9
writing mit3 connectivity input metanalysis files for drug BRD-A08658758
BRD-K00640357 6 of 68
drug BRD-K00640357 processed in 0.7
writing mit3 connectivity input metanalysis files for drug BRD-K00640357
BRD-K11533227 7 of 68
drug BRD-K11533227 processed in 0.8
writing mit3 connectivity input metanalysis files for drug BRD-K11533227
BRD-K16410418 8 of 68
dru

## Convert disease MITHrIL output into Connectivity Score input

In [8]:
mith_file=MITH_OUT_DISEASE+DISEASE+'_mith3.perturbations.txt'
cs_data=mith_to_cs_in(mith_file)


In [10]:
cs_data

,gene_id,gene,Perturbation,p.value,adj.p.value
0,6197,"RPS6KA3, CLS, HU-3, ISPK-1, MAPKAPK1B, MRX19, ...",0.386794,1.000000,1.0
1,chebi:456216,ADP(3-),316.216558,0.819590,1.0
2,6196,"RPS6KA2, HU-2, MAPKAPK1C, RSK, RSK3, S6K-alpha...",0.384756,1.000000,1.0
3,chebi:30616,ATP(4-),56.183675,0.908046,1.0
4,5594,"MAPK1, ERK, ERK-2, ERK2, ERT1, MAPK2, NS13, P4...",1.009191,1.000000,1.0
...,...,...,...,...,...
388518,gl:G00711,(GlcA)1 (GlcNAc)2,0.337279,0.592704,1.0
388519,gl:G00526,(GlcA)2 (GlcNAc)2,1.862948,0.282359,1.0
390472,cpd:C00999,Ferrocytochrome b5,0.184327,1.000000,1.0
390474,123956252,RIPPLY2-CYB5R4,0.039842,1.000000,1.0


In [12]:
# write CS input
cs_input_name=DISEASE+'_mith3_signature.csv'
write_cs_input(cs_data, cs_input_name, CS_IN_DISEASE)

Connectivity score disease input file for MITHrIL perturbation data written at G:\My Drive\unict\unict 2024-25\netcos\connectivity_score\input\disease_signature_2025\LIHC\ 
file name: LIHC_mith3_signature.csv


# <div align="center"><b>Calculate connectivity score</b>

source code: connectivity_score.py

In [ ]:
from connectivity_score import 